<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;

class Program
{
    public static void Main(string[] args)
    {
        OnlineOrder onlineOrder = new OnlineOrder(1, DateTime.Now, "Valteil", "valteil@fearandhunger.com", "Credit Card", "Standard");
        PhysicalOrder physicalOrder = new PhysicalOrder(2, DateTime.Now, "John Doe", "123 Main St, Syrodil", "Express", "1234567890");
        SpecializedOrder specializedOrder = new SpecializedOrder(3, DateTime.Now, "Jane Doe", "Discount 10%", "Fragile items", 2);
        InternationalOrder internationalOrder = new InternationalOrder(4, DateTime.Now, "Foreign Customer", "foreign@example.com", "PayPal", "Canada", "Customs info", "TRACK12345");
        DiscountedOrder discountedOrder = new DiscountedOrder(5, DateTime.Now, "Discount Customer", 15, "DISCOUNT15", 500);

        onlineOrder.AddItem(new Item("Deadric heart", 1200));
        physicalOrder.AddItem(new Item("Minotaur testicles", 800));
        specializedOrder.AddItem(new Item("Wagic wand", 500));
        internationalOrder.AddItem(new Item("Maple syrup", 200));
        discountedOrder.AddItem(new Item("Discounted item", 1000));

        physicalOrder.RemoveItem(new Item("Minotaur testicles", 800));

        Console.WriteLine($"Online Order Total: {onlineOrder.CalculateTotal()}");
        Console.WriteLine($"Physical Order Total: {physicalOrder.CalculateTotal()}");
        Console.WriteLine($"Specialized Order Total: {specializedOrder.CalculateTotal()}");
        Console.WriteLine($"International Order Total: {internationalOrder.CalculateTotal()}");
        Console.WriteLine($"Discounted Order Total: {discountedOrder.CalculateTotal()}");

        OrderManager<Order> orderManager = new OrderManager<Order>();
        orderManager.OrderProcessed += OrderManager_OrderProcessed;
        orderManager.AddOrder(onlineOrder);
        orderManager.AddOrder(physicalOrder);
        orderManager.AddOrder(specializedOrder);
        orderManager.AddOrder(internationalOrder);
        orderManager.AddOrder(discountedOrder);

        orderManager.PrintAllOrders();
        orderManager.GetOrdersByStatus("Pending").ToList().ForEach(order => Console.WriteLine($"Order {order.OrderId} is pending."));
    }

    private static void OrderManager_OrderProcessed(Order order, EventArgs e)
    {
        Console.WriteLine($"Order {order.OrderId} has been processed.");
    }
}

public class Item
    {
        public string Name { get; set; }
        public decimal Price { get; set; }

        public Item(string name, decimal price)
        {
            Name = name;
            Price = price;
        }
    }

public class Order
{
    public int OrderId { get; set; }
    public DateTime CreationDate { get; set; }
    public decimal TotalAmount { get; set; }
    public string OrderStatus { get; set; }
    protected List<Item> Items { get; set; }
    public string CustomerName { get; set; }
    public DateTime ShippingDate { get; set; }

    public event EventHandler StatusChanged;

    public Order(int orderId, DateTime creationDate, string customerName)
    {
        OrderId = orderId;
        CreationDate = creationDate;
        Items = new List<Item>();
        OrderStatus = "Pending";
        CustomerName = customerName;
    }

    public virtual decimal CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in Items)
        {
            TotalAmount += item.Price;
        }
        return TotalAmount;
    }

    public virtual void AddItem(Item item)
    {
        Items.Add(item);
    }

    public virtual void RemoveItem(Item item)
    {
        Items.Remove(item);
    }

    public void UpdateStatus(string newStatus)
    {
        OrderStatus = newStatus;
        OnStatusChanged();
        Console.WriteLine($"Order {OrderId} status updated to {newStatus}");
    }

    protected virtual void OnStatusChanged()
    {
        StatusChanged?.Invoke(this, EventArgs.Empty);
    }

    public virtual void PrintOrderDetails()
    {
        Console.WriteLine($"Order ID: {OrderId}, Customer: {CustomerName}, Status: {OrderStatus}, Total: {CalculateTotal()}");
    }

    public virtual void TrackShipping()
    {
        ShippingDate = DateTime.Now;
        UpdateStatus("Shipped");
    }
}

public class OnlineOrder : Order
{
    public string CustomerEmail { get; set; }
    public string PaymentMethod { get; set; }
    public string ShippingMethod { get; set; }

    public OnlineOrder(int orderId, DateTime creationDate, string customerName, string customerEmail, string paymentMethod, string shippingMethod)
        : base(orderId, creationDate, customerName)
    {
        CustomerEmail = customerEmail;
        PaymentMethod = paymentMethod;
        ShippingMethod = shippingMethod;
    }

    public override void AddItem(Item item)
    {
        base.AddItem(item);
        Console.WriteLine($"Item '{item.Name}' added to online order. Delivery information will be sent to {CustomerEmail}.");
    }

    public void AddItem(Item item, string note)
    {
        base.AddItem(item);
        Console.WriteLine($"Item '{item.Name}' added to online order with note: {note}. Delivery information will be sent to {CustomerEmail}.");
    }

    public void SendConfirmationEmail()
    {
        Console.WriteLine($"Confirmation email sent to {CustomerEmail} for order {OrderId}.");
    }

    public void SendShippingNotification()
    {
        Console.WriteLine($"Shipping notification sent to {CustomerEmail} for order {OrderId} via {ShippingMethod}.");
    }
}

public class PhysicalOrder : Order
{
    public string DeliveryAddress { get; set; }
    public string DeliveryMethod { get; set; }
    public string ContactNumber { get; set; }

    public PhysicalOrder(int orderId, DateTime creationDate, string customerName, string deliveryAddress, string deliveryMethod, string contactNumber)
        : base(orderId, creationDate, customerName)
    {
        DeliveryAddress = deliveryAddress;
        DeliveryMethod = deliveryMethod;
        ContactNumber = contactNumber;
    }

    public override void RemoveItem(Item item)
    {
        base.RemoveItem(item);
        Console.WriteLine($"Item '{item.Name}' removed from physical order. Return information will be sent to {DeliveryAddress}.");
    }

    public void RemoveItem(Item item, string reason)
    {
        base.RemoveItem(item);
        Console.WriteLine($"Item '{item.Name}' removed from physical order due to {reason}. Return information will be sent to {DeliveryAddress}.");
    }

    public void ScheduleDelivery()
    {
        Console.WriteLine($"Delivery scheduled for order {OrderId} to {DeliveryAddress} using {DeliveryMethod}.");
    }

    public void ScheduleDeliveryCall()
    {
        Console.WriteLine($"Delivery call scheduled for order {OrderId} to {ContactNumber}.");
    }
}

public class SpecializedOrder : Order
{
    public string SpecialConditions { get; set; }
    public string SpecialInstructions { get; set; }
    public int PriorityLevel { get; set; }

    public SpecializedOrder(int orderId, DateTime creationDate, string customerName, string specialConditions, string specialInstructions, int priorityLevel)
        : base(orderId, creationDate, customerName)
    {
        SpecialConditions = specialConditions;
        SpecialInstructions = specialInstructions;
        PriorityLevel = priorityLevel;
    }

    public override decimal CalculateTotal()
    {
        decimal total = base.CalculateTotal();
        if (SpecialConditions.Contains("Discount"))
        {
            total *= 0.9m;
        }
        return total;
    }

    public void ApplySpecialInstructions()
    {
        Console.WriteLine($"Special instructions '{SpecialInstructions}' applied to order {OrderId}.");
    }

    public void PrioritizeOrder()
    {
        PriorityLevel = 1;
        Console.WriteLine($"Order {OrderId} has been prioritized to level {PriorityLevel}.");
    }
}

public class InternationalOrder : OnlineOrder
{
    public string Country { get; set; }
    public string CustomsInformation { get; set; }
    public string TrackingNumber { get; set; }

    public InternationalOrder(int orderId, DateTime creationDate, string customerName, string customerEmail, string paymentMethod, string country, string customsInformation, string trackingNumber)
        : base(orderId, creationDate, customerName, customerEmail, paymentMethod, "International")
    {
        Country = country;
        CustomsInformation = customsInformation;
        TrackingNumber = trackingNumber;
    }

    public override void AddItem(Item item)
    {
        base.AddItem(item);
        Console.WriteLine($"Item '{item.Name}' added to international order. Customs information will be sent to {CustomerEmail} for {Country}.");
    }

    public void ProcessCustoms()
    {
        Console.WriteLine($"Customs information '{CustomsInformation}' processed for order {OrderId} to {Country}.");
    }

    public void TrackInternationalShipment()
    {
        Console.WriteLine($"Tracking number {TrackingNumber} for order {OrderId} is being tracked.");
    }
}

public interface IDiscountable
{
    decimal ApplyDiscount(decimal total);
}

public class DiscountedOrder : Order, IDiscountable
{
    public decimal DiscountPercentage { get; set; }
    public string DiscountCode { get; set; }
    public decimal MinimumPurchase { get; set; }

    public DiscountedOrder(int orderId, DateTime creationDate, string customerName, decimal discountPercentage, string discountCode, decimal minimumPurchase)
        : base(orderId, creationDate, customerName)
    {
        DiscountPercentage = discountPercentage;
        DiscountCode = discountCode;
        MinimumPurchase = minimumPurchase;
    }

    decimal IDiscountable.ApplyDiscount(decimal total)
    {
        return total * (1 - DiscountPercentage / 100);
    }

    public override decimal CalculateTotal()
    {
        decimal total = base.CalculateTotal();
        return ((IDiscountable)this).ApplyDiscount(total);
    }

    public void ApplyDiscountCode()
    {
        Console.WriteLine($"Discount code '{DiscountCode}' applied to order {OrderId}.");
    }

    public bool CheckQualification()
    {
        if (CalculateTotal() >= MinimumPurchase)
        {
            Console.WriteLine($"Order {OrderId} qualifies for discount.");
            return true;
        }
        else
        {
            Console.WriteLine($"Order {OrderId} does not qualify for discount.");
            return false;
        }
    }
}

public class OrderManager<T> where T : Order
{
    private List<T> orders = new List<T>();

    public delegate void OrderProcessingHandler(Order order, EventArgs e);
    public event OrderProcessingHandler OrderProcessed;

    public void AddOrder(T order)
    {
        orders.Add(order);
        OnOrderProcessed(order);
    }

    protected virtual void OnOrderProcessed(Order order)
    {
        OrderProcessed?.Invoke(order, EventArgs.Empty);
    }

    public void PrintAllOrders()
    {
        foreach (var order in orders)
        {
            Console.WriteLine($"Order ID: {order.OrderId}, Status: {order.OrderStatus}, Total: {order.CalculateTotal()}");
        }
    }

    public IEnumerable<T> GetOrdersByStatus(string status)
    {
        return orders.Where(o => o.OrderStatus == status);
    }
}